In [1]:
import os
from tqdm import tqdm
import cv2
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from numpy import asarray

In [2]:
longitud, altura = 150, 150
modelo = './baseline/baseline.h5'
pesos_modelo = './baseline/pesos.h5'
cnn = load_model(modelo)
cnn.load_weights(pesos_modelo)

In [3]:
def Dataset_loader(DIR, RESIZE):
  IMG = []
  read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
  for IMAGE_NAME in tqdm(os.listdir(DIR)):
    PATH = os.path.join(DIR, IMAGE_NAME)
    _, ftype = os.path.splitext(PATH)
    if (ftype == ".jpg" or ftype=='.jpeg'):
      img = read(PATH)
      img = cv2.resize(img, (RESIZE, RESIZE))
      IMG.append(np.array(img) / 255.)
  return IMG

In [4]:
def Dataset_loader2(DIR, RESIZE):
  IMG = []
  for IMAGE_NAME in tqdm(os.listdir(DIR)):
    PATH = os.path.join(DIR, IMAGE_NAME)
    _, ftype = os.path.splitext(PATH)
    if (ftype == ".jpg" or ftype=='.jpeg'):
      image = Image.open(PATH).resize((RESIZE, RESIZE))
      data = asarray(image)
      IMG.append(np.array(data) / 255.)
  return IMG

In [ ]:
train_benign = np.array(Dataset_loader('./DermMel/train_sep/NotMelanoma', altura))
train_malign = np.array(Dataset_loader('./DermMel/train_sep/Melanoma', altura))


benign_train_label = np.zeros(len(train_benign))
malign_train_label = np.ones(len(train_malign))

X_train = np.concatenate((train_benign, train_malign), axis = 0)
Y_train = np.concatenate((benign_train_label, malign_train_label), axis = 0)

 67%|██████▋   | 3557/5341 [00:06<00:02, 620.59it/s]

In [ ]:
y_pred = cnn.predict(X_train)
print("Done.")

In [36]:
print(y_pred[0][0])

0.9999994


In [14]:
#Segundo conjunto de imágenes
train_benign2 = np.array(Dataset_loader('./input/skin-cancer-malignant-vs-benign/train/benign', altura))
train_malign2 = np.array(Dataset_loader('./input/skin-cancer-malignant-vs-benign/train/malignant', altura))


benign_train_label2 = np.zeros(len(train_benign2))
malign_train_label2 = np.ones(len(train_malign2))

X_train2 = np.concatenate((train_benign2, train_malign2), axis = 0)
Y_train2 = np.concatenate((benign_train_label2, malign_train_label2), axis = 0)

100%|██████████| 1197/1197 [00:02<00:00, 449.41it/s]


In [15]:
y_pred4 = cnn.predict(X_train2)
print("Done.")

Done.


In [37]:
def ValidationTraining(y_pred,Y_test):
  print(len(y_pred))
  for j in range(50,0,-2):
      predictions = []
      truePositives = 0
      falsePositives = 0
      trueNegatives = 0
      falseNegatives = 0
      for i in range(len(y_pred)):
        if y_pred[i][1] > j/100:
          predictions.append(1)
          if Y_test[i]==1:
              truePositives+=1
          else: falsePositives+=1
        else:
          predictions.append(0)
          if Y_test[i]==0:
              trueNegatives+=1
          else: falseNegatives+=1

      sensitivity=truePositives/(truePositives+falseNegatives)
      especificity=trueNegatives/(trueNegatives+falsePositives)
      accuracy=accuracy_score(Y_test, predictions)
      print("ClassificationNumber:",j/100)
      print("Accuracy model: ",accuracy)
      print("Sensitivity: ",sensitivity)
      print("Especificity: ",especificity)
      print("TP",truePositives,"FN",falseNegatives,"TN",trueNegatives,"FP",falsePositives)
      print("-----------------------------------------------------------------")

In [38]:
ValidationTraining(y_pred,Y_train)

10682
ClassificationNumber: 0.5
Accuracy model:  0.6933158584534731
Sensitivity:  0.6594270735817263
Especificity:  0.72720464332522
TP 3522 FN 1819 TN 3884 FP 1457
-----------------------------------------------------------------
ClassificationNumber: 0.48
Accuracy model:  0.6950945515821008
Sensitivity:  0.6657929226736566
Especificity:  0.7243961804905449
TP 3556 FN 1785 TN 3869 FP 1472
-----------------------------------------------------------------
ClassificationNumber: 0.46
Accuracy model:  0.6948137052986332
Sensitivity:  0.6699120014978468
Especificity:  0.7197154090994196
TP 3578 FN 1763 TN 3844 FP 1497
-----------------------------------------------------------------
ClassificationNumber: 0.44
Accuracy model:  0.6965923984272608
Sensitivity:  0.6775884665792923
Especificity:  0.7155963302752294
TP 3619 FN 1722 TN 3822 FP 1519
-----------------------------------------------------------------
ClassificationNumber: 0.42
Accuracy model:  0.6976221681333084
Sensitivity:  0.683579

In [39]:
ValidationTraining(y_pred4,Y_train2)

2637
ClassificationNumber: 0.5
Accuracy model:  1.0
Sensitivity:  1.0
Especificity:  1.0
TP 1197 FN 0 TN 1440 FP 0
-----------------------------------------------------------------
ClassificationNumber: 0.48
Accuracy model:  1.0
Sensitivity:  1.0
Especificity:  1.0
TP 1197 FN 0 TN 1440 FP 0
-----------------------------------------------------------------
ClassificationNumber: 0.46
Accuracy model:  1.0
Sensitivity:  1.0
Especificity:  1.0
TP 1197 FN 0 TN 1440 FP 0
-----------------------------------------------------------------
ClassificationNumber: 0.44
Accuracy model:  1.0
Sensitivity:  1.0
Especificity:  1.0
TP 1197 FN 0 TN 1440 FP 0
-----------------------------------------------------------------
ClassificationNumber: 0.42
Accuracy model:  1.0
Sensitivity:  1.0
Especificity:  1.0
TP 1197 FN 0 TN 1440 FP 0
-----------------------------------------------------------------
ClassificationNumber: 0.4
Accuracy model:  1.0
Sensitivity:  1.0
Especificity:  1.0
TP 1197 FN 0 TN 1440 FP 0
-

In [19]:
#TESTING

In [5]:
#Para este modelo se ha encontrado que el mejor resultado de corte es 0.08
def ValidationTesting(y_pred,Y_test):
  classificationNumber=0.08
  predictions = []
  truePositives = 0
  falsePositives = 0
  trueNegatives = 0
  falseNegatives = 0
  for i in range(len(y_pred)):
    if y_pred[i][1] > classificationNumber:
      predictions.append(1)
      if Y_test[i]==1:
          truePositives+=1
      else: falsePositives+=1
    else:
      predictions.append(0)
      if Y_test[i]==0:
          trueNegatives+=1
      else: falseNegatives+=1

  sensitivity=truePositives/(truePositives+falseNegatives)
  especificity=trueNegatives/(trueNegatives+falsePositives)
  accuracy=accuracy_score(Y_test, predictions)
  print("ClassificationNumber:",classificationNumber)
  print("Accuracy model: ",accuracy)
  print("Sensitivity: ",sensitivity)
  print("Especificity: ",especificity)
  print("TP",truePositives,"FN",falseNegatives,"TN",trueNegatives,"FP",falsePositives)
  print("-----------------------------------------------------------------")

In [19]:
import random
random.seed(30)

test_benign = np.array(Dataset_loader('./DermMel/test/NotMelanoma', altura))
test_malign = np.array(Dataset_loader('./DermMel/test/Melanoma', altura))


benign_test_label = np.zeros(len(test_benign))
malign_test_label = np.ones(len(test_malign))

X_test = np.concatenate((test_benign, test_malign), axis = 0)
Y_test = np.concatenate((benign_test_label, malign_test_label), axis = 0)

random.shuffle(X_test)
random.shuffle(Y_test)

X_experimentsArray=np.array_split(X_test, 13)
Y_experimentsArray=np.array_split(Y_test, 13)

X_13E=[]
Y_13E=[]

print(len(test_benign),len(test_malign))
rangeNumber=0
for i in range(0,13):
    #100 imagenes benignas
    benign=(test_benign[rangeNumber:rangeNumber+100])
    zeros=(np.zeros(100))
    #100 imagenes malignas
    malign=(test_malign[rangeNumber:rangeNumber+100])
    ones=(np.ones(100))
    
    rangeNumber+=100
    print(rangeNumber)
    X_13E.append(np.concatenate((benign,malign),axis=0))
    Y_13E.append(np.concatenate((zeros,ones),axis=0))  


100%|██████████| 1781/1781 [00:04<00:00, 425.94it/s]


1780 1781
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


In [8]:
#Predicción (solo se ejecuta una vez)
y_pred2 = cnn.predict(X_test)
print("Done.")

Done.


In [9]:
print(len(X_test),len(Y_test),len(malign_test_label),len(benign_test_label),len(test_benign),len(test_malign),y_pred2[5][0],Y_test[5])

3561 3561 1781 1780 1780 1781 0.080419675 1.0


In [20]:
predictionsArray=[]
for experiment in X_13E:
   predictionsArray.append(cnn.predict(experiment)) 

In [21]:
print(len(X_13E),len(Y_13E),len(predictionsArray))
for i in range(len(predictionsArray)):
    print("Experimento ",i)
    print(len(predictionsArray[i]),len(Y_13E[i]))
    ValidationTesting(predictionsArray[i],Y_13E[i])
    print("---------------------------------------------")

13 13 13
Experimento  0
200 200
ClassificationNumber: 0.08
Accuracy model:  0.745
Sensitivity:  0.79
Especificity:  0.7
TP 79 FN 21 TN 70 FP 30
-----------------------------------------------------------------
---------------------------------------------
Experimento  1
200 200
ClassificationNumber: 0.08
Accuracy model:  0.675
Sensitivity:  0.78
Especificity:  0.57
TP 78 FN 22 TN 57 FP 43
-----------------------------------------------------------------
---------------------------------------------
Experimento  2
200 200
ClassificationNumber: 0.08
Accuracy model:  0.665
Sensitivity:  0.78
Especificity:  0.55
TP 78 FN 22 TN 55 FP 45
-----------------------------------------------------------------
---------------------------------------------
Experimento  3
200 200
ClassificationNumber: 0.08
Accuracy model:  0.71
Sensitivity:  0.81
Especificity:  0.61
TP 81 FN 19 TN 61 FP 39
-----------------------------------------------------------------
---------------------------------------------
E

In [23]:
#Segundo dataset de testing
import random
random.seed(30)

test_benign2 = np.array(Dataset_loader2('./input/skin-cancer-malignant-vs-benign/test/benign', altura))
test_malign2 = np.array(Dataset_loader2('./input/skin-cancer-malignant-vs-benign/test/malignant', altura))

benign_test_label2 = np.zeros(len(test_benign2))
malign_test_label2 = np.ones(len(test_malign2))

X_test2 = np.concatenate((test_benign2, test_malign2), axis = 0)
Y_test2 = np.concatenate((benign_test_label2, malign_test_label2), axis = 0)

X_13E_2=[]
Y_13E_2=[]

print(len(test_malign2),len(test_benign2))
rangeNumber=0
for i in range(0,13):
    #20 imagenes benignas
    benign=(test_benign2[rangeNumber:rangeNumber+20])
    zeros=(np.zeros(20))
    #20 imagenes malignas
    malign=(test_malign2[rangeNumber:rangeNumber+20])
    ones=(np.ones(20))
    
    rangeNumber+=20
    print(rangeNumber)
    X_13E_2.append(np.concatenate((benign,malign),axis=0))
    Y_13E_2.append(np.concatenate((zeros,ones),axis=0))
    

100%|██████████| 300/300 [00:00<00:00, 535.74it/s]


300 360
20
40
60
80
100
120
140
160
180
200
220
240
260


In [ ]:
y_pred3 = cnn.predict(X_test2)
print("Done.")

In [ ]:
ValidationTesting(y_pred3,Y_test2)

In [24]:
predictionsArray2=[]
for experiment in X_13E_2:
   predictionsArray2.append(cnn.predict(experiment)) 

In [25]:
print(len(X_13E_2[1]),len(Y_13E_2[1]),len(predictionsArray2[1]))
for i in range(len(predictionsArray2)):
    print("Experimento ",i)
    print(len(predictionsArray2[i]),len(Y_13E_2[i]))
    ValidationTesting(predictionsArray2[i],Y_13E_2[i])
    print("---------------------------------------------")

40 40 40
Experimento  0
40 40
ClassificationNumber: 0.08
Accuracy model:  0.775
Sensitivity:  0.85
Especificity:  0.7
TP 17 FN 3 TN 14 FP 6
-----------------------------------------------------------------
---------------------------------------------
Experimento  1
40 40
ClassificationNumber: 0.08
Accuracy model:  0.825
Sensitivity:  0.9
Especificity:  0.75
TP 18 FN 2 TN 15 FP 5
-----------------------------------------------------------------
---------------------------------------------
Experimento  2
40 40
ClassificationNumber: 0.08
Accuracy model:  0.8
Sensitivity:  0.85
Especificity:  0.75
TP 17 FN 3 TN 15 FP 5
-----------------------------------------------------------------
---------------------------------------------
Experimento  3
40 40
ClassificationNumber: 0.08
Accuracy model:  0.875
Sensitivity:  0.95
Especificity:  0.8
TP 19 FN 1 TN 16 FP 4
-----------------------------------------------------------------
---------------------------------------------
Experimento  4
40 40